In [6]:
import random
import csv
import time
from scipy.stats import expon
import numpy as np
from simulation import generate_population, inverse_mapping, update_healthy_state, terminate, update_state
import matplotlib.pyplot as plt

In [61]:
def simulate(S, T):
    population = generate_population(S)
    grid = inverse_mapping(population)
    break_t = T
    death = 0
    total_infection = 0
    max_infection = 0
    t_max_inf = 0
    for t in range(T):
        current_inf = sum([1 for i in population if i.infection])
        if current_inf > max_infection:
            t_max_inf = t
            max_infection = current_inf
        population = update_healthy_state(population)
        if terminate(population):
            break_t = t
            break
        update_state(population, grid)
    death = 1000 - len(population)
    total_infection = death + sum([1 for i in population if i.infection or i.immunity])
    return break_t, death, total_infection, max_infection, t_max_inf

In [4]:
R = 10000
M = 1000
T = 500
S_interval = 0.25
S_list = [i*S_interval for i in range(int((1+S_interval)/S_interval))]
print(S_list)

[0.0, 0.25, 0.5, 0.75, 1.0]


In [63]:
for S in S_list:
    with open("result_%.2f.csv" % S,"w") as csvfile: 
        writer = csv.writer(csvfile)
        writer.writerow(["break_t", "death rate","total infection rate","max infection rate", "t max infection"])
        st = time.time()
        for run in range(R):
            break_t, death, total_infection, max_infection, t_max_inf = simulate(S, T)
            death_rate, total_infection_rate, max_infection_rate = death/M, total_infection/M, max_infection/M
            writer.writerow([break_t, death_rate, total_infection_rate, max_infection_rate, t_max_inf])
            if not run % 50:
                print(run)
    print("cost time for S=%.2f %.2f" % (S, time.time() - st))

0
cost time for S=0.00 11.63
0
cost time for S=0.25 5.64
0
cost time for S=0.50 1.48
0
cost time for S=0.75 0.50
0
cost time for S=1.00 0.88


In [21]:
def print_his(values, name, S, test=None):
    plt.rcParams['figure.dpi'] = 300
    num_bins = 100
    ax = plt.axes()
    values, base, patches = plt.hist(values, num_bins)
    if test:
        x_label = name + ' when S = %.2f' % S + ' with testing rate = %.2f' % test
    else:
        x_label = name + ' when S = %.2f' % S
    plt.xlabel(x_label)
    plt.ylabel('Count')
    ax_bis = ax.twinx()
    values = np.append(values, 0)
    ax_bis.plot(base, np.cumsum(values)/ np.cumsum(values)[-1], color='darkorange', marker='o', linestyle='-', markersize = 1, label = "Cumulative Histogram" )
    plt.ylabel("Cumulative probability")
#     plt.show()
    plt.savefig(x_label + '.png', dpi=300)
    plt.clf()

In [25]:
def print_statistics(values, name, S, test=None):
    if test:
        print("Average " + name + " is %.4f" % np.average(values) + ' when S = %.2f' % S + ' with testing rate = %.2f' % test)
    else:
        print("Average " + name + " is %.4f" % np.average(values) + ' when S = %.2f' % S)
    return np.average(values)

In [22]:
for S in [0, 0.25, 0.5, 0.75]:
    for test in [0.01, 0.05, 0.1]:
        break_ts, death_rates, total_infection_rates, max_infection_rates, t_max_infs = [], [], [], [], []
        with open("result_%.2f_%.2f.csv" % (S, test),"r") as csvfile:
            csv_read = csv.DictReader(csvfile)
            for line in csv_read:
                break_ts.append(float(line['break_t']))
                death_rates.append(float(line['death rate']))
                total_infection_rates.append(float(line['total infection rate']))
                max_infection_rates.append(float(line['max infection rate']))
                t_max_infs.append(float(line['t max infection']))
        print_his(death_rates, "Death rate", S, test)
        print_his(total_infection_rates, "Total infection rate", S, test)
        print_his(max_infection_rates, "Max infection rate", S, test)
        print_statistics(death_rates, "Death rate", S, test)
        print_statistics(break_ts, "Break t", S, test)
        print_statistics(total_infection_rates, "Total infection rate", S, test)
        print_statistics(max_infection_rates, "Max infection rate", S, test)
        print_statistics(t_max_infs, "t max infection", S, test)
    

Average Death rate is 0.0314 when S = 0.00 with testing rate = 0.01
Average Break t is 429.3365 when S = 0.00 with testing rate = 0.01
Average Total infection rate is 0.6326 when S = 0.00 with testing rate = 0.01
Average Max infection rate is 0.0443 when S = 0.00 with testing rate = 0.01
Average t max infection is 61.0216 when S = 0.00 with testing rate = 0.01
Average Death rate is 0.0099 when S = 0.00 with testing rate = 0.05
Average Break t is 183.6682 when S = 0.00 with testing rate = 0.05
Average Total infection rate is 0.1999 when S = 0.00 with testing rate = 0.05
Average Max infection rate is 0.0246 when S = 0.00 with testing rate = 0.05
Average t max infection is 36.7755 when S = 0.00 with testing rate = 0.05
Average Death rate is 0.0020 when S = 0.00 with testing rate = 0.10
Average Break t is 48.2816 when S = 0.00 with testing rate = 0.10
Average Total infection rate is 0.0408 when S = 0.00 with testing rate = 0.10
Average Max infection rate is 0.0170 when S = 0.00 with testin

<Figure size 1800x1200 with 0 Axes>

In [26]:
avg_death_rates = []
avg_break_ts = []
avg_total_infection_rates = []
avg_max_infection_rats = []
avg_t_max_infs = []
for S in [0, 0.25, 0.5, 0.75, 1]:
    break_ts, death_rates, total_infection_rates, max_infection_rates, t_max_infs = [], [], [], [], []
    with open("result_%.2f.csv" % S,"r") as csvfile:
        csv_read = csv.DictReader(csvfile)
        for line in csv_read:
            break_ts.append(float(line['break_t']))
            death_rates.append(float(line['death rate']))
            total_infection_rates.append(float(line['total infection rate']))
            max_infection_rates.append(float(line['max infection rate']))
            t_max_infs.append(float(line['t max infection']))
#     print_his(death_rates, "Death rate", S)
#     print_his(total_infection_rates, "Total infection rate", S)
#     print_his(max_infection_rates, "Max infection rate", S)
    avg_death_rates.append(print_statistics(death_rates, "Death rate", S))
    avg_break_ts.append(print_statistics(break_ts, "Break t", S))
    avg_total_infection_rates.append(print_statistics(total_infection_rates, "Total infection rate", S))
    avg_max_infection_rats.append(print_statistics(max_infection_rates, "Max infection rate", S))
    avg_t_max_infs.append(print_statistics(t_max_infs, "t max infection", S))

Average Death rate is 0.0362 when S = 0.00
Average Break t is 454.8314 when S = 0.00
Average Total infection rate is 0.7259 when S = 0.00
Average Max infection rate is 0.0520 when S = 0.00
Average t max infection is 59.2217 when S = 0.00
Average Death rate is 0.0145 when S = 0.25
Average Break t is 269.0856 when S = 0.25
Average Total infection rate is 0.2919 when S = 0.25
Average Max infection rate is 0.0267 when S = 0.25
Average t max infection is 52.4791 when S = 0.25
Average Death rate is 0.0022 when S = 0.50
Average Break t is 62.4268 when S = 0.50
Average Total infection rate is 0.0444 when S = 0.50
Average Max infection rate is 0.0150 when S = 0.50
Average t max infection is 13.1760 when S = 0.50
Average Death rate is 0.0007 when S = 0.75
Average Break t is 24.4654 when S = 0.75
Average Total infection rate is 0.0144 when S = 0.75
Average Max infection rate is 0.0109 when S = 0.75
Average t max infection is 6.0494 when S = 0.75
Average Death rate is 0.0004 when S = 1.00
Average 

In [30]:
def print_s_plot(values, name, S_list):
    plt.rcParams['figure.dpi'] = 300
    plt.xlabel("S")
    plt.ylabel(name)
    plt.plot(S_list, values)
    plt.savefig(name + '.png', dpi=300)
    plt.clf()

In [31]:
print(avg_death_rates)
print(avg_total_infection_rates)
print(avg_max_infection_rats)

print_s_plot(avg_death_rates, "death rates", S_list)
print_s_plot(avg_total_infection_rates, "total infection rates", S_list)
print_s_plot(avg_max_infection_rats, "max infection rates", S_list)

[0.036163400000000005, 0.014521200000000001, 0.0022314000000000006, 0.0007194, 0.0003987]
[0.7259114999999999, 0.29185089999999997, 0.0443544, 0.014355200000000002, 0.008000000000000004]
[0.05197920000000002, 0.026659300000000007, 0.015027200000000003, 0.0109011, 0.008000000000000004]


<Figure size 1800x1200 with 0 Axes>